In [1]:
from sec_processing.utils import *


17:31:31.278 Logging initialized with level INFO


In [4]:
import numpy as np

def calculate_cost_of_debt(ticker):
    """
    Calculates after-tax cost of debt for a given ticker
    using Income Statement and Balance Sheet data.

    :param ticker: Stock ticker symbol (str)
    :return: cost of debt (float) or np.nan if data missing
    """
    # Fetch Income Statement and Balance Sheet
    income_df = fetch_report_to_df(ticker, "income_statement")
    balance_df = fetch_report_to_df(ticker, "balance_sheet")

    try:
        # Transpose for easier column selection (matching your pipeline)
        income_df_t = income_df.transpose()
        balance_df_t = balance_df.transpose()

        # --- Extract Interest Expense ---
        # Adjust column names as per your data schema
        interest_expense_cols = [col for col in income_df_t.columns if "interest" in col.lower() and "expense" in col.lower()]
        if not interest_expense_cols:
            raise ValueError("Interest Expense column not found")
        interest_expense = income_df_t[interest_expense_cols[0]].mean()

        # --- Extract Pre-Tax Income and Income Tax Expense ---
        pretax_income_cols = [col for col in income_df_t.columns if "pretax" in col.lower() or "pre-tax" in col.lower()]
        income_tax_expense_cols = [col for col in income_df_t.columns if "income tax" in col.lower() and "expense" in col.lower()]

        if not pretax_income_cols or not income_tax_expense_cols:
            raise ValueError("Pre-Tax Income or Income Tax Expense column not found")

        pretax_income = income_df_t[pretax_income_cols[0]].mean()
        income_tax_expense = income_df_t[income_tax_expense_cols[0]].mean()

        # --- Calculate Effective Tax Rate ---
        tax_rate = income_tax_expense / pretax_income if pretax_income != 0 else 0

        # --- Calculate Total Debt ---
        # Adjust with your hybrid mapping + regex later
        total_debt = 0
        debt_cols = [col for col in balance_df_t.columns if "debt" in col.lower()]
        if debt_cols:
            total_debt = balance_df_t[debt_cols].sum(axis=1).mean()
        else:
            raise ValueError("Debt columns not found")

        # --- Final Cost of Debt Calculation ---
        cost_of_debt = (interest_expense / total_debt) * (1 - tax_rate)

        return cost_of_debt

    except Exception as e:
        print(f"Could not calculate cost of debt for {ticker}: {e}")
        return np.nan

# Example usage:
tickers = ["MSFT", "TSLA", "GOOGL"]
for ticker in tickers:
    cod = calculate_cost_of_debt(ticker)
    print(f"{ticker} Cost of Debt: {cod:.4f}")


Could not calculate cost of debt for MSFT: Interest Expense column not found
MSFT Cost of Debt: nan
Could not calculate cost of debt for TSLA: 'NoneType' object has no attribute 'lower'
TSLA Cost of Debt: nan
Could not calculate cost of debt for GOOGL: Interest Expense column not found
GOOGL Cost of Debt: nan
